## Create dataset with documents ids
The goal of this notebook is to merge all documents into single one and add the `document id` property to every document.

In [8]:
from pprint import pprint

import hashlib
import pandas as pd
import os

### Read all data files and merge them into a single one.

In [9]:
source_faqs_directory = "/home/jovyan/work/mage/data/faqs/"
doc_with_ids_path = "/home/jovyan/work/notebook/retrieval_evaluation/dataset_with_doc_ids.csv"

### Add a document id field
The length of the field is the same, as pipeline uses to index data.

In [10]:
fileset = []
column_names=["source_system", "category", "question", "answer"]

for file_name in os.listdir(source_faqs_directory):
    if file_name.endswith(".csv"):
        data = pd.read_csv(os.path.join(source_faqs_directory, file_name), delimiter=";", names=column_names, skiprows=1, usecols=(0,1,2,3))
        fileset.append(data)

df = pd.concat(fileset, ignore_index=True)
pprint(df[:1].to_dict(orient="records"))

[{'answer': 'Der Analyseprozess bei Engel & Völkers Digital Invest basiert auf '
            'einem speziell entwickelten Konzept für die Projektfinanzierung '
            'von Bauvorhaben und Grundstücken. Zunächst erfolgt eine interne '
            'Analyse, die auf dem umfangreichen Know-how des Engel & '
            'Völkers-Netzwerks aufbaut, einschließlich Research-Material und '
            'persönlicher Kontakte vor Ort. Um eine unabhängige Validierung zu '
            'gewährleisten, werden zusätzlich externe Experten hinzugezogen: '
            'Die bulwiengesa appraisal GmbH führt eine Markt- und '
            'Standortanalyse durch, CBRE übernimmt die technische '
            'Projektanalyse, Mazars analysiert den Projektentwickler, und HUTH '
            'DIETRICH HAHN prüft die rechtlichen Aspekte. Basierend auf diesen '
            'Gutachten erstellt Engel & Völkers Digital Invest '
            'eigenverantwortlich die finale Projekteinschätzung. Dieser '
            'u

In [11]:
for idx, row in df.iterrows():
    hex_digest = hashlib.md5(row["question"].encode() + row["answer"].encode()).hexdigest()
    id = hex_digest[:12]
    df.at[idx, "document_id"] = id

df[:1]

,source_system,section,question,answer,document_id
0,evdi,Analysekonzept,Wie läuft der Analyseprozess für Immobilienpro...,Der Analyseprozess bei Engel & Völkers Digital...,f2624f5125f9


### Save the final dataframe

In [12]:
df.to_csv(doc_with_ids_path, sep=";", index=False)